In [1]:
import tensorflow as tf

In [12]:
dataset = tf.data.Dataset.range(10)


In [13]:
print(dataset)

<RangeDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>


In [14]:
dataset = dataset.window(5,shift = 1,drop_remainder = True)

In [15]:
dataset = dataset.flat_map(lambda window:window.batch(5))

In [16]:
dataset = dataset.map(lambda window: (window[:-1],window[-1:]))
for x,y in dataset:
  print(x.numpy(),y.numpy())

[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[5 6 7 8] [9]


In [17]:
dataset = dataset.shuffle(buffer_size = 10)
dataset = dataset.batch(2).prefetch(1)
for x, y in dataset:
  print('x = ',x.numpy())
  print('y = ',y.numpy())

x =  [[3 4 5 6]
 [2 3 4 5]]
y =  [[7]
 [6]]
x =  [[4 5 6 7]
 [5 6 7 8]]
y =  [[8]
 [9]]
x =  [[0 1 2 3]
 [1 2 3 4]]
y =  [[4]
 [5]]


In [18]:
def trend(time, slope=0):
 return slope * time
def seasonal_pattern(season_time):
 return np.where(season_time < 0.4,
 np.cos(season_time * 2 * np.pi),
 1 / np.exp(3 * season_time))
def seasonality(time, period, amplitude=1, phase=0):
 season_time = ((time + phase) % period) / period
 return amplitude * seasonal_pattern(season_time)

In [20]:
import numpy as np
def noise(time, noise_level=1, seed=None):
 rnd = np.random.RandomState(seed)
 return rnd.randn(len(time)) * noise_level
time = np.arange(4 * 365 + 1, dtype="float32")
series = trend(time, 0.1)
baseline = 10
amplitude = 20
slope = 0.09
noise_level = 5

In [21]:
series = baseline + trend(time, slope)
series += seasonality(time, period=365, amplitude=amplitude)
series += noise(time, noise_level, seed=42)

In [22]:
def windowed_dataset(series, window_size,
 batch_size, shuffle_buffer):
 dataset = tf.data.Dataset.from_tensor_slices(series)
 dataset = dataset.window(window_size + 1, shift=1,
 drop_remainder=True)
 dataset = dataset.flat_map(lambda window:
 window.batch(window_size + 1))
 dataset = dataset.shuffle(shuffle_buffer).map(
 lambda window:
 (window[:-1], window[-1]))
 dataset = dataset.batch(batch_size).prefetch(1)
 return dataset


In [23]:
split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]
window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

In [24]:
dataset = windowed_dataset(x_train, window_size, batch_size,
 shuffle_buffer_size)

In [25]:
model = tf.keras.models.Sequential([
 tf.keras.layers.Dense(10, input_shape=[window_size],
 activation="relu"),
 tf.keras.layers.Dense(10, activation="relu"),
 tf.keras.layers.Dense(1)
])


In [27]:
model.compile(loss="mse",optimizer=tf.keras.optimizers.SGD(
 learning_rate=1e-6,
 momentum=0.9))

In [28]:
model.fit(dataset,epochs=100,verbose=1)


Epoch 1/100
31/31 [==============================] - 1s 3ms/step - loss: 1096.0731
Epoch 2/100
31/31 [==============================] - 0s 3ms/step - loss: 85.2613
Epoch 3/100
31/31 [==============================] - 0s 3ms/step - loss: 61.6901
Epoch 4/100
31/31 [==============================] - 0s 3ms/step - loss: 59.6057
Epoch 5/100
31/31 [==============================] - 0s 3ms/step - loss: 57.9547
Epoch 6/100
31/31 [==============================] - 0s 3ms/step - loss: 56.6838
Epoch 7/100
31/31 [==============================] - 0s 3ms/step - loss: 55.7997
Epoch 8/100
31/31 [==============================] - 0s 3ms/step - loss: 54.8564
Epoch 9/100
31/31 [==============================] - 0s 3ms/step - loss: 53.2078
Epoch 10/100
31/31 [==============================] - 0s 3ms/step - loss: 52.3994
Epoch 11/100
31/31 [==============================] - 0s 4ms/step - loss: 51.5537
Epoch 12/100
31/31 [==============================] - 0s 3ms/step - loss: 50.5650
Epoch 13/100
31/31 [===

In [29]:
print(model.predict(series[1000:1020][np.newaxis]))

[[103.724266]]
